<a href="https://colab.research.google.com/github/LenSin3/big-data-challenge/blob/main/cloud_etl_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [37]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-03 05:43:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... failed: Connection timed out.
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|2001:4800:3e1:1::228|:443... failed: Cannot assign requested address.
Retrying.

--2021-02-03 05:43:45--  (try: 2)  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... failed: Connection timed out.
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|2001:4800:3e1:1::228|:443... failed: Cannot assign requested address.
Retrying.

--2021-02-03 05:44:19--  (try: 3)  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... failed: Connection timed out.
Connecting to jdbc.postgresql.org (jdbc

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ToysReviews").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz'
spark.sparkContext.addFile(url)
toys_review_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Toys_v1_00.tsv.gz'), sep=r'\t', header=True)

# Show DataFrame
toys_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [ ]:
# Count number of rows in dataset
num_rows = toys_review_df.count()
print(f'The are {num_rows} records in the dataset')

The are 4864249 records in the dataset


In [ ]:
# Create data frame for overall reviews
# Create dataframe with the following columns review_id, customer_id, product_id, product_parent, review_date
review_df = toys_review_df.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date')
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RDIJS7QYB6XNR|   18778586|B00EDBY7X8|     122952789| 2015-08-31|
|R36ED1U38IELG8|   24769659|B00D7JFOPC|     952062646| 2015-08-31|
| R1UE3RPRGCOLD|   44331596|B002LHA74O|     818126353| 2015-08-31|
|R298788GS6I901|   23310293|B00ARPLCGY|     261944918| 2015-08-31|
|  RNX4EXOBBPN5|   38745832|B00UZOPOFW|     717410439| 2015-08-31|
|R3BPETL222LMIM|   13394189|B009B7F6CA|     873028700| 2015-08-31|
|R3SORMPJZO3F2J|    2749569|B0101EHRSM|     723424342| 2015-08-31|
|R2RDOJQ0WBZCF6|   41137196|B00407S11Y|     383363775| 2015-08-31|
|R2B8VBEPB4YEZ7|     433677|B00FGPU7U2|     780517568| 2015-08-31|
|R1CB783I7B0U52|    1297934|B0013OY0S0|     269360126| 2015-08-31|
| R2D90RQQ3V8LH|   52006292|B00519PJTW|     493486387| 2015-08-31|
|R1Y4ZOUGFMJ327|   32071052|B001TCY2DO|     459122467| 2015-08

In [ ]:
# Create data frame for products
products_df = toys_review_df.select('product_id', 'product_title').drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IANTUPU|Lego, Star Wars, ...|
|B002TMK962|Rock - Concert Ti...|
|B00AOTTSQY|Adult & Children ...|
|B00CH3O5S6|48 Wholesale Wood...|
|B00MF3SAF2|Teenitor 3.7V 680...|
|B004D0C5WY|ThinkMax Silver B...|
|B00NHQHJZS|LEGO Technic Arct...|
|B00NQJ2DBG|Super Bright Long...|
|B0082IGA06|Yu-Gi-Oh! - Dark ...|
|1609581520|American Girl Car...|
|B004RGYVLC|DC Comics POP Her...|
|B0053EX3D4|Inuyasha Kirara T...|
|B00OTLHNI8|Bloco Toys T-Rex ...|
|B00BXEYI8W|Kid Galaxy Robot ...|
|B00JUG2NSM|Weiss Schwarz Kil...|
|B00LVPY252|Maxx Action 30" T...|
|B00IGRHWSC|Mattel Hot Wheels...|
|B00U6FU3YM|Educational Insig...|
|B00Z7FS3Q6|Beach Ball Set, 3...|
|B000WEAGTY|Inflatable Tradit...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import min, max, count

In [ ]:
# Create data frame for customers
customers_df = toys_review_df.groupBy('customer_id').count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   49997672|    2|
|   27435626|    2|
|   50312871|   24|
|   43801871|    2|
|   33212984|    2|
|   41500251|    4|
|   11234247|    2|
|   36739867|    8|
|   31341684|   65|
|    2668626|    1|
|   31325270|    5|
|   15293494|    1|
|    7549374|    1|
|   24268456|    2|
|    2529063|    1|
|   48244313|    2|
|     255431|    1|
|   50583551|   17|
|   24453085|    1|
|   21596603|    2|
+-----------+-----+
only showing top 20 rows



In [ ]:
# Update column name from count to customer_count
customers_df_renamed = customers_df.withColumnRenamed('count','customer_count')
customers_df_renamed.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   49997672|             2|
|   27435626|             2|
|   50312871|            24|
|   43801871|             2|
|   33212984|             2|
|   41500251|             4|
|   11234247|             2|
|   36739867|             8|
|   31341684|            65|
|    2668626|             1|
|   31325270|             5|
|   15293494|             1|
|    7549374|             1|
|   24268456|             2|
|    2529063|             1|
|   48244313|             2|
|     255431|             1|
|   50583551|            17|
|   24453085|             1|
|   21596603|             2|
+-----------+--------------+
only showing top 20 rows



In [ ]:
mode = "append"
jdbc_url = "jdbc:postgresql://mypostgresdb.cbwo1b8joelu.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "lenpk85awsrdspostgres", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_df.write.jdbc(url=jdbc_url, table='toys_review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored